In [14]:
from glob import glob
import pandas as pd
import os
from tqdm import tqdm as tqdm 
from IPython.display import Image
from pathlib import Path
import json
import numpy as np

In [19]:
results_root = "/storage/czw/btbench/single_electrode_btbench_full/eval_results_full_SS_SM/linear_voltage_single_electrode"
lite_results_root = "/storage/czw/linear_voltage_single_electrode_SS_SM_prototype/linear_voltage_single_electrode"


In [16]:
def get_test_results(results, eval_name):
    eval_results = results["evaluation_results"]
    assert len(eval_results.keys()) == 1
    records = []
    for subject_trial in eval_results:
        subject = subject_trial.split("_")[0][len("btbank"):]
        trial = subject_trial.split("_")[1]
        subject_trial_results = eval_results[subject_trial]['electrode']
        for electrode in subject_trial_results:
            electrode_results = subject_trial_results[electrode]
            time_bin_results = electrode_results["time_bins"]
            for time_bin in time_bin_results:
                time_bin_start = time_bin['time_bin_start']
                fold_results = time_bin['folds']
                avg_test = np.mean([f['test_roc_auc'] for f in fold_results])
                records.append({
                    "subject": subject,
                    "trial": trial,
                    "ID": f'{electrode}-sub_{subject}',
                    "electrode": electrode,
                    "avg_test": avg_test,
                    "time_bin": time_bin_start,
                    "task": eval_name
                })
        return records

In [24]:
def get_results_df(results_root):
    all_records = []
    all_results_paths = glob(os.path.join(results_root, "*"))
    for path in tqdm(all_results_paths):
        name = Path(path).stem
        sub_id = name.split("_")[1][len("btbank"):]
        trial_id = name.split("_")[2]
        eval_name = "_".join(name.split("_")[3:])
        with open(path, "r") as f:
            results = json.load(f)
            test_results = get_test_results(results, eval_name)
        all_records += test_results
    results_df = pd.DataFrame.from_records(all_records)
    return results_df

In [25]:
results_df = get_results_df(results_root)

100%|██████████████████████████████████████████████████████████████████████████████████| 494/494 [00:34<00:00, 14.14it/s]


In [35]:
avg_results_df = results_df.groupby(['ID', 'task']).max('avg_test')

In [26]:
lite_results_df = get_results_df(lite_results_root)

100%|██████████████████████████████████████████████████████████████████████████████████| 228/228 [00:19<00:00, 11.58it/s]


In [34]:
avg_lite_results_df = lite_results_df.groupby(['ID', 'task']).max('avg_test')

In [39]:
(avg_lite_results_df['avg_test'] - avg_results_df['avg_test']).dropna().mean()

0.0021242491143563997

avg_test  time_bin
ID            task                                 
F10Fa1-sub_10 delta_pitch        0.522710     1.125
              delta_volume       0.526442     1.125
              frame_brightness   0.522776     1.125
              global_flow        0.518618     1.125
              global_flow_angle  0.523075     1.125
...                                   ...       ...
T3bOT9-sub_1  volume             0.521583     1.125
              word_gap           0.523281     1.125
              word_head_pos      0.520881     1.125
              word_index         0.519237     1.125
              word_length        0.517072     1.125

[28683 rows x 2 columns]